## GSAT trend patterns

In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [ ]:
# load data
dir_HadCRUT5 = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure1/'
HadCRUT5 = xr.open_dataset(dir_HadCRUT5 + 'tas_HadCRUT5_annual_anomalies_processed.nc')

dir1 ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supplementary/S1/data/'
HadCRUT5_forced = xr.open_dataset(dir1 + 'HadCRUT_Forced_signal.nc')
HadCRUT5_internal = xr.open_dataset(dir1 + 'HadCRUT_residual.nc')

In [ ]:
HadCRUT5_forced

In [ ]:
HadCRUT5

#### define function

In [ ]:
variable_name = ['10yr', '20yr', '30yr', '40yr', '50yr', '60yr', '70yr']
# define the function
def separate_data_into_intervals(data, start_year, end_year, time_interval):
    """
    This function is used to separate the data into different time intervals
    """
    # create a dictionary to store the data
    data_dict = {}
    for i in range(len(variable_name)):
        # calculate the start year and end year
        start_year = time_interval[variable_name[i]][0]
        end_year = time_interval[variable_name[i]][1]
        # select the data
        data_dict[variable_name[i]] = data.sel(year=slice(str(start_year), str(end_year)))
    return data_dict

In [ ]:
time_interval = {
    "10yr":(2013,2022),
    "20yr":(2003,2022),
    "30yr":(1993,2022),
    "40yr":(1983,2022),
    "50yr":(1973,2022),
    "60yr":(1963,2022),
    "70yr":(1953,2022)
}
start_year = 1950
end_year   = 2022

In [ ]:
data_dict = separate_data_into_intervals(HadCRUT5, start_year, end_year, time_interval)
forced_dict = separate_data_into_intervals(HadCRUT5_forced, start_year, end_year, time_interval)
internal_dict = separate_data_into_intervals(HadCRUT5_internal, start_year, end_year, time_interval)

In [ ]:
# 1st calculate the origianl trend 
trend_dict = {}
pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = data_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    trend_dict[variable_name[i]] = slope
    pvalue_dict[variable_name[i]] = p_values

In [ ]:
trend_annual_np = {}
pvalue_annual_np = {}

for i in range(len(variable_name)):
    trend_annual_np[variable_name[i]] = trend_dict[variable_name[i]].values
    pvalue_annual_np[variable_name[i]] = pvalue_dict[variable_name[i]].values
    
trend_annual_np['10yr']

In [ ]:
trend_annual_da = {}
pvalue_annual_da = {}

for interval, data in trend_annual_np.items():
    trend_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})
for interval, data in pvalue_annual_np.items():
    pvalue_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})

In [ ]:
# # save the data into netcdf file
dir_output = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure1/data_revision/'
for interval, data in trend_annual_da.items():
    data.to_netcdf(dir_output + 'Raw_HadCRUT5_annual_' + interval + '_trend.nc')
    
for interval, data in pvalue_annual_da.items():
    data.to_netcdf(dir_output + 'Raw_HadCRUT5_annual_' + interval + '_p_value.nc')

In [ ]:
# 2nd calculate the forced trend
forced_trend_dict = {}
forced_pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = forced_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    forced_trend_dict[variable_name[i]] = slope
    forced_pvalue_dict[variable_name[i]] = p_values

forced_trend_annual_np = {}
forced_pvalue_annual_np = {}

for i in range(len(variable_name)):
    forced_trend_annual_np[variable_name[i]] = forced_trend_dict[variable_name[i]].values
    forced_pvalue_annual_np[variable_name[i]] = forced_pvalue_dict[variable_name[i]].values
    
forced_trend_annual_da = {}
forced_pvalue_annual_da = {}

for interval, data in forced_trend_annual_np.items():
    forced_trend_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})
for interval, data in forced_pvalue_annual_np.items():
    forced_pvalue_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})

In [ ]:
# # save the data into netcdf file
# for interval, data in forced_trend_annual_da.items():
#     data.to_netcdf(dir_output + 'HadCRUT5_annual_forced_' + interval + '_trend.nc')
    
# for interval, data in forced_pvalue_annual_da.items():
#     data.to_netcdf(dir_output + 'HadCRUT5_annual_forced_' + interval + '_p_value.nc')

In [ ]:
# 3rd calculate the internal trend
internal_trend_dict = {}
internal_pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = internal_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    internal_trend_dict[variable_name[i]] = slope
    internal_pvalue_dict[variable_name[i]] = p_values

internal_trend_annual_np = {}
internal_pvalue_annual_np = {}

for i in range(len(variable_name)):
    internal_trend_annual_np[variable_name[i]] = internal_trend_dict[variable_name[i]].values
    internal_pvalue_annual_np[variable_name[i]] = internal_pvalue_dict[variable_name[i]].values
    
internal_trend_annual_da = {}
internal_pvalue_annual_da = {}

for interval, data in internal_trend_annual_np.items():
    internal_trend_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})
for interval, data in internal_pvalue_annual_np.items():
    internal_pvalue_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": data_dict[interval].lat, "lon": data_dict[interval].lon})


In [ ]:
# # # save the data into netcdf file
# # dir_output = '/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/1950_2022_results/Trend_data/'
# for interval, data in internal_trend_annual_da.items():
#     data.to_netcdf(dir_output + 'HadCRUT5_annual_internal_' + interval + '_trend.nc')
    
# for interval, data in internal_pvalue_annual_da.items():
#     data.to_netcdf(dir_output + 'HadCRUT5_annual_internal_' + interval + '_p_value.nc')

### Plotting with the Robinson Projections

In [ ]:
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.pad_inches'] = 0.1
plt.rcParams['savefig.transparent'] = True

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

def plot_trend_with_significance(trend_data, lats, lons, p_values, GMST_p_values=None, levels=None, extend=None, cmap=None, 
                                 title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - trend_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """

    # Create a new figure/axis if none is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(20, 15), subplot_kw={'projection': ccrs.Robinson()})
        ax.set_global()
  
    # Determine significance mask (where p-values are less than 0.05)
    # significance_mask = p_values < 0.05
    insignificance_mask = p_values >= 0.05
    
    # Plotting
    # contour_obj = ax.pcolormesh(lons, lats, trend_data,  cmap='RdBu_r',vmin=-5.0, vmax=5.0, transform=ccrs.PlateCarree(central_longitude=180), shading='auto')
    contour_obj = ax.contourf(lons, lats, trend_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree(central_longitude=0))

    # Plot significance masks with different hatches
    ax.contourf(lons, lats, insignificance_mask, levels=[0, 0.05, 1.0], hatches=[None,'///'], colors='none', edgecolor = 'white', transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      color='gray', alpha=0.35, linestyle='--')

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 16}
    gl.ylabel_style = {'size': 16}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    # ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj


In [ ]:
# define an asymmetric colormap
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

intervals = [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 1.1, 1.3]

# Normalizing the intervals to [0, 1]
min_interval = min(intervals)
max_interval = max(intervals)
normalized_intervals = [(val - min_interval) / (max_interval - min_interval) for val in intervals]

# cmap = mcolors.ListedColormap(palettable.scientific.diverging.Vik_20.mpl_colors)
cmap=mcolors.ListedColormap(palettable.cmocean.diverging.Balance_20.mpl_colors)

# Define the colors at each interval
colors = [(0.00784313725490196, 0.2, 0.4627450980392157, 1.0),
    (0.00784313725490196, 0.2, 0.4627450980392157, 1.0),
    (0.023529411764705882, 0.32941176470588235, 0.5450980392156862, 1.0),
    (0.023529411764705882, 0.32941176470588235, 0.5450980392156862, 1.0),
    (1.0, 1.0, 1.0, 1.0),
    (1.0, 0.9, 0.98, 1.0),
    (1.0, 0.8, 0.5, 1.0),
    (1.0, 0.803921568627451, 0.607843137254902, 1.0), 
    (1.0, 0.6000000000000001, 0.20000000000000018, 1.0),
    (1.0, 0.4039215686274509, 0.0, 1.0),
    (0.8999999999999999, 0.19999999999999996, 0.0, 1.0),
    (0.7470588235294118, 0.0, 0.0, 1.0), 
    (0.6000000000000001, 0.0, 0.0, 1.0),
    (0.44705882352941173, 0.0, 0.0, 1.0),
    (0.30000000000000004, 0.0, 0.0, 1.0),
    (0.14705882352941177, 0.0, 0.0, 1.0),
    (0.0, 0.0, 0.0, 1.0)]

# Creating a list of tuples with normalized positions and corresponding colors
color_list = list(zip(normalized_intervals, colors))

# Create the colormap
custom_cmap = LinearSegmentedColormap.from_list('my_custom_cmap', color_list)

# Create a normalization
norm = Normalize(vmin=min_interval, vmax=max_interval)

In [ ]:
# transform the trend data unit to degree per decade
for i in range(len(variable_name)):
    trend_annual_da[variable_name[i]] = trend_annual_da[variable_name[i]] * 10
    forced_trend_annual_da[variable_name[i]] = forced_trend_annual_da[variable_name[i]] * 10
    internal_trend_annual_da[variable_name[i]] = internal_trend_annual_da[variable_name[i]] * 10

In [ ]:
# check the min and max value of the trend
for i in range(len(variable_name)):
    print(variable_name[i], trend_annual_da[variable_name[i]].min().values, trend_annual_da[variable_name[i]].max().values)
    print(variable_name[i], forced_trend_annual_da[variable_name[i]].min().values, forced_trend_annual_da[variable_name[i]].max().values)
    print(variable_name[i], internal_trend_annual_da[variable_name[i]].min().values, internal_trend_annual_da[variable_name[i]].max().values)

In [ ]:
lat = internal_trend_annual_da['10yr']['lat'].values
lon = internal_trend_annual_da['10yr']['lon'].values


titles_rows = ["a. 10yr(2013-2022)", "b. 20yr(2003-2022)", "c. 30yr(1993-2022)", "d. 40yr(1983-2022)", "e. 50yr(1973-2022)", 
                  "f. 60yr(1963-2022)", "g. 70yr(1953-2022)"]

titles = ["Raw", "Forced wrt. MMEM", "Unforced wrt. MMEM"] # only show the title for the first row
# plot the Raw forced and unforced trend for each time interval
# 7 rows and 3 columns
"""
with each row representing a time interval;
and each column representing the raw, forced and unforced trend respectively
"""

levels = np.arange(-1.0, 1.1, 0.1)

# Define the GridSpec
fig = plt.figure(figsize=(15, 25))
gs = gridspec.GridSpec(7, 3, figure=fig, wspace=0.01, hspace=0.1)

periods = ["10yr", "20yr", "30yr", "40yr", "50yr", "60yr", "70yr"]
extend = 'both'
axes = {}
for i, period in enumerate(periods):
    for j in range(3):
        is_left = j==0
        is_bottom_row = i>=6
        ax = fig.add_subplot(gs[i, j], projection=ccrs.Robinson(180))
        ax.set_global()
        
        axes[(i,j)]=ax
        # if i == 0:
            # ax.text(0.5, 1.25, titles[j], va='bottom', ha='center', rotation='horizontal', fontsize=16, 
                    # weight='bold',transform=ax.transAxes)
            
        if j == 0:
            trend_data = trend_annual_da[period]
            trend_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
            p_values = pvalue_annual_da[period]
            p_values_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(p_values, coord=lon)
    
            contour_obj = plot_trend_with_significance(trend_with_cyclic, lat, lon_with_cyclic, p_values_with_cyclic, 
                        GMST_p_values=None, levels=levels,extend=extend, cmap=cmap, 
                        title=" ", ax=ax, show_xticks = is_bottom_row, show_yticks = is_left)
        elif j == 1:
            trend_data = forced_trend_annual_da[period]
            trend_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
            p_values = forced_pvalue_annual_da[period]
            p_values_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(p_values, coord=lon)
    
            contour_obj1 = plot_trend_with_significance(trend_with_cyclic, lat, lon_with_cyclic, p_values_with_cyclic, 
                        GMST_p_values=None, levels=levels,extend=extend, cmap=cmap, 
                        title=" ", ax=ax, show_xticks = is_bottom_row, show_yticks = is_left)
        else:
            trend_data = internal_trend_annual_da[period]
            trend_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(trend_data, coord=lon)
            p_values = internal_pvalue_annual_da[period]
            p_values_with_cyclic, lon_with_cyclic = cutil.add_cyclic_point(p_values, coord=lon)
    
            contour_obj2 = plot_trend_with_significance(trend_with_cyclic, lat, lon_with_cyclic, p_values_with_cyclic, 
                        GMST_p_values=None, levels=levels,extend=extend, cmap=cmap, 
                        title=" ", ax=ax, show_xticks = is_bottom_row, show_yticks = is_left)
# add the title for each row
# for i, period in enumerate(periods):
#     axes[i,0].text(0.125, 1.1, titles_rows[i], va='bottom', ha='center', rotation='horizontal', fontsize=16, 
#                     weight='bold',transform=axes[i, 0].transAxes)

# Add horizontal colorbars
cbar_ax = fig.add_axes([0.2, 0.08, 0.6, 0.01])
cbar = plt.colorbar(contour_obj, cax=cbar_ax, orientation='horizontal', extend=extend)
fig.text(0.5, 0.06,'Annual SAT trend ($^\circ$C/decade)',fontsize=22, ha='center', va='bottom')
cbar.ax.tick_params(labelsize=18)

plt.tight_layout()
fig.savefig('Segemented-HadCRUT5-OLS-trend-pattern-separation-with-Non-sig95%.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
client.close()
scluster.close()